In [ ]:
'''
Written  by: Jeffrey Blitt
Modified on: 01/13/19
'''

# Import all necessary libraries for the program to run
import RPi.GPIO as GPIO  # Regular pin input/output package for Pi
from time import sleep   # Sleep function for musical timing
from lxml import html    # Retrieval of HTML from a URL
import re                # Regular Expression package
import requests          # For web content decryption
GPIO.setwarnings(False)  # Ignore warnings
GPIO.setmode(GPIO.BCM)   # Set pin reference mode to BCM

# Constants
BUZZ1PIN = 2             # Declare the pin used for output to the buzzer
BEAT = .25               # Declare the length of one beat

# Declare the frequency of each note in octave zero (using A 440 tuning)
noteDict = {
    "C" : 16.35,
    "C#": 17.32,
    "Db": 17.32,
    "D" : 18.35,
    "D#": 19.45,
    "Eb": 19.45,
    "E" : 20.60,
    "F" : 21.83,
    "F#": 23.12,
    "Gb": 23.12,
    "G" : 24.50,
    "G#": 25.96,
    "Ab": 25.96,
    "A" : 27.50,
    "A#": 29.14,
    "Bb": 29.14,
    "B" : 30.87,
    ""  :     0
}

# Declare the characters allowed in a note string
allowedChars = ['A','B','C','D','E','F','G','b','#',' ','_','.','^','*']

In [ ]:
def note_to_freq(note):
    '''
    Purpose:    Convert a music note name to the corresponding frequency in Hz
    Parameters: note (string) - Ex: "Bb3"
    Returns:    freq (float) - Ex: 233.12
    '''
    # Take a blank note to be 0 Hz
    if note == "":
        octave = 0
        freq   = 0
    # If an octave is specified
    elif str(note)[-1].isdigit() == True:
        # Read the last character of the note as the octave number
        octave = int(str(note)[-1])
        # Remove the octave number from the note
        note   = note[:-1]
        # With the note and octave known, determine the correct frequency
        freq   = float(noteDict.get(note)) * (2 ** octave)
    # If no octave is specified, default to that of middle C
    else:
        octave = 4
        # Determine the frequency of the note in octave 4
        freq   = float(noteDict.get(note)) * (2 ** octave)
    return freq


def get_html(url):
    '''
    Purpose:    Retrieve a string of decoded HTML from a certain URL
    Parameters: url (string) - Ex: "https://noobnotes.net/super-mario-bros-theme-nintendo/?solfege=false"
    Returns:    html (string) - Ex: "<!DOCTYPE html>\r\n<html lang="en-GB" xmlns[...]"
    '''
    # Retrieve page content
    page = requests.get(url)
    # Decode the page content with UTF-8
    html = page.content.decode("utf-8")
    return html


def strip_tags(html):
    '''
    Purpose:    Remove all html tags (like "<div>") from the html string
    Parameters: html (string) - Ex: "<!DOCTYPE html>\r\n<html lang="en-GB" xmlns[...]"
    Returns:    cleanHtml (string) - Ex: "\r\n[...]"
    '''
    # Take out the shortest substring starting with '<' and ending
    # with '>' until there are no more remining in the HTML string
    cleanHtml = re.sub('<[^<]+?>', '', html)
    return cleanHtml


def parse_html_text(html):
    '''   
    Purpose:    Parse music from a given string of HTML from "noobnotes.net"
    Parameters: html (string) - Ex: "<!DOCTYPE html>\r\n<html lang="en-GB" xmlns[...]"
    Returns:    notesList (list) - Ex: "['_','^E ^E ^E','^C ^E ^G G','_','^C G E',[...]]"
    '''
    html = str(html)
    # Trim the HTML string so that it starts at the beginning of the song
    rawScraped = html.split('<div class="post-content">')[1]
    # Remove the HTML tags from the string
    cleanMusic = strip_tags(rawScraped)
    # Replace and remove various blankspace and notation characters
    cleanMusic = cleanMusic.replace('\xa0', '')
    cleanMusic = cleanMusic.replace('&nbsp;', '_')
    cleanMusic = cleanMusic.replace('-', ' ')
    cleanMusic = cleanMusic.lstrip()
    # Split the string at each newline character
    songTextArray = cleanMusic.split('\n')
    notesList = []
    # Store the notes in an array of lines
    for line in songTextArray:
        # Ignore blank lines
        if line == '':
            break
        # Ignore any improperly parsed lines with space at the beginning
        elif line[0] == ' ':
            break
        else:
            # Replace any multiple spaces with one space
            while '  ' in line:
                line = line.replace('  ', ' ')
            lineLength = len(line)
            # If all characters in the line are allowed, store the notes
            for charIndex in range(lineLength):
                if line[charIndex] in allowedChars:
                    if charIndex == lineLength - 1:
                        notesList.append(line)
                    else: continue
                else: break
    return notesList



'''
Class used to initialize and control a piezo buzzer
'''
class Buzzer:
    # Usage of constructor: "<buzzerName> = Buzzer(<pin>)"
    def __init__(self, buzzerPin):
        # Initialize input/output through the specified pin
        self.pin = buzzerPin
        GPIO.setup(self.pin, GPIO.IN)
        GPIO.setup(self.pin, GPIO.OUT)
    def __del__(self): 
        class_name = self.__class__.__name__
        # Clean up pin input/output when Buzzer object is terminated
        GPIO.cleanup
    

    def stop_tone(self):
        '''  
        Purpose:    Stop the buzzer from emitting sound
        Parameters: None
        Returns:    None
        '''
        # Stop output to the buzzer pin
        GPIO.PWM(self.pin, 0.1).stop()
    

    def play_tone(self, freq):
        '''  
        Purpose:    Play a tone at the given frequency on the buzzer
        Parameters: freq (float) - Ex: 233.12
        Returns:    None
        '''
        # If given a rest, wait for 3/4 of a beat
        if freq == 0:
            sleep(BEAT * .75)
        else:
            # Declare width modulation output at the given frequency
            tone = GPIO.PWM(self.pin, freq)
            # Start the output at a duty cycle of 50
            tone.start(50)
            # Wait for 3/4 of a beat
            sleep(BEAT * .75)
            # Stop the audio output
            tone.stop()


    def play_note(self, note):
        '''  
        Purpose:    Play a given note on the buzzer
        Parameters: note (string) - Ex: Bb3
        Returns:    None
        '''
        # Convert the given note to its corresponding frequency
        freq = note_to_freq(note)
        # Play the determined tone
        self.play_tone(freq)
    

    def play_song(self, songArray):
        '''  
        Purpose:    Play a given song on the buzzer
        Parameters: songArray (list)
        Returns:    None
        '''
        # Iterate through the music by line
        for line in songArray:
            # Split each line into an array of notes, dividing at spaces
            line = line.split(' ')
            # Print the line as it is played
            print(line)
            # Interpret notes as they are notated on the webpage
            for rawNote in line:
                note = "C"
                octave = 4
                if rawNote == '_':
                    sleep(BEAT)
                    continue
                elif rawNote == '':
                    continue
                elif rawNote[0] == '^':
                    octave += 1
                    note = rawNote[1:]
                elif rawNote[0] == '*':
                    octave += 2
                    note = rawNote[1:]
                elif rawNote[0] == '.':
                    octave -= 1
                    note = rawNote[1:]
                elif rawNote[0] == '_':
                    octave -= 2
                    note = rawNote[1:]
                else: note = rawNote
                note = note + str(octave)
                # Play the interpreted note
                self.play_note(note)
                # Rest for the last 1/4 of the beat
                sleep(BEAT * .25)
                self.stop_tone()
            # Rest at the end of each line to separate phrases
            sleep(BEAT)

In [ ]:
# Example usage
songs = ['https://noobnotes.net/how-to-save-a-life-the-fray/?solfege=false',
         'https://noobnotes.net/super-mario-bros-theme-nintendo/?solfege=false',
         'https://noobnotes.net/happy-birthday-traditional/?solfege=false',
         'https://noobnotes.net/bohemian-rhapsody-queen/?solfege=false&transpose=0',
         'https://noobnotes.net/let-it-go-frozen-disney/?solfege=false',
         'https://noobnotes.net/tiny-dancer-elton-john/?solfege=false',
         'https://noobnotes.net/eleanor-rigby-the-beatles/?solfege=false',
         'https://noobnotes.net/colors-wind-pocahontas-disney/?solfege=false',
         'https://noobnotes.net/all-star-smash-mouth/?solfege=false',
         'https://noobnotes.net/here-comes-the-sun-the-beatles/?solfege=false']
buzzer = Buzzer(BUZZ1PIN)
rawHtml = get_html(songs[4])

musicArray = parse_html_text(rawHtml)
try:
    buzzer.play_song(musicArray)
except KeyboardInterrupt:
    buzzer.stop_tone()

Note: It is recommended to stop the kernel, restart it, and run each cell in order each time a new song is played. 